## Навигатор
* [1. Подключение необходимых библиотек и данных](#lib)
* [2. Загрузка данных в Data Frame](#load)
* [3. Приведение и оптимизация типов](#types)
* [4. EDA](#eda)
    * [4.1. Основные статистики](#eda_def)
    * [4.2. Исследование вещественных признаков](#eda_num_features)
    * [4.3. Исследование номинативных признаков](#eda_nom)
    * [4.4. Исследование целевой переменной](#eda_target)
* [5. Обработка выбросов и пропусков](#out_nan)
    * [5.1. Вещественные признаки](#out_nan_def)
    * [5.2. Номинативные признаки](#out_nan_mat)
* [6. Классы подготовки данных](#data_prepare)
* [7. Разбиение на train и test](#split)
* [8. Построение модели](#modeling)
* [9. Прогнозирование на тестовом датасете](#prediction)

## 1. Подключение библиотек и скриптов <a class='anchor' id='lib'>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
matplotlib.rcParams.update({'font.size': 10})

In [ ]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()
    

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### **Пути к директориям и файлам**

In [ ]:
TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/test.csv'

## 2. Загрузка данных в Data Frame <a class='anchor' id='load'>

### Описание файлов
- **train.csv** - тренировочные данные
- **test.csv** - тестовые данные
- **sampleSubmission.csv** - образец файла для отправки в правильном формате

### Описание датасета:

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры

In [ ]:
# Визуально знакомимся с тренировочными данными
train_df = pd.read_csv(TRAIN_DATASET_PATH)
train_df.tail()

In [ ]:
# Визуально знакомимся с тестовыми данными
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.tail()

## **3. Приведение и оптимизация типов** <a class='anchor' id='types'>

In [ ]:
reduce_mem_usage(train_df)

In [ ]:
train_df.dtypes

In [ ]:
train_df['Id'] = train_df['Id'].astype(str)
train_df['DistrictId'] = train_df['DistrictId'].astype(str)

## 4. EDA <a class='anchor' id='eda'>

### 4.1. Основные статистики <a class='anchor' id='eda_def'>

In [ ]:
train_df.describe()

In [ ]:
# Процентное количество пропущенных значений
train_df.isnull().mean() * 100

## Вывод по основным статистикам:
* Rooms, Square, LifeSquare, KitchenSquare, HouseFloor имеют аномально низкие значения близкие и/или равные нулю
* Заметны выбросы максимальных значений у таких признаков как Rooms, Square, LifeSquare, KitchenSquare, HouseYear
* Есть пропущенные значения у признаков LifeSquare, Healthcare_1


### **4.2 Исследование вещественных признаков** <a class='anchor' id='eda_num_features'>

In [ ]:
df_num_features = train_df.select_dtypes(include=['int8', 'int16', 'int32', 'float32'])
df_num_features.drop('Price', axis=1, inplace=True)
df_num_features.hist(figsize=(25, 25), bins=45, grid=False);

### **4.3. Исследование номинативных признаков** <a class='anchor' id='eda_nom'>

In [ ]:
df_nom_features = train_df.select_dtypes(include=['object', 'category'])

for feature in df_nom_features:
    print('Количество уникальных значений признака ' + feature + '\n')
    print(train_df[feature].value_counts())
    print()

### Выводы по номинативным признакам

#### DistrictId
Аномалий не наблюдается.
Вся территория делится на 205 районов,минимальное значение 0 может говоить о том, что индексация начинается с 0.

#### Ecology_2
Больше всего зданий относится к классу B

#### Ecology_3
Больше всего зданий относится к классу B

#### Shops_2
Типы магазинов поблизости. Больше всего магазинов типа B.

### 4.4. Исследование целевой переменной <a class='anchor' id='eda_target'>

In [ ]:
plt.figure(figsize = (12, 8))

train_df['Price'].hist(bins=45, grid=False)
plt.ylabel('Count')
plt.xlabel('Price')

plt.title('Target distribution')
plt.show()

## 5. Обработка выбросов и пропусков <a class='anchor' id='out_nan'>

### 5.1. Вещественные признаки <a class='anchor' id='out_nan_def'>

### Rooms

In [ ]:
train_df['Rooms'].value_counts()

In [ ]:
# Обработаем выбросы и нулевые значения комнат
train_df.loc[train_df['Rooms'] == 0, 'Rooms'] = 1
train_df.loc[train_df['Rooms'] >= 6, 'Rooms'] = train_df['Rooms'].median()

### Square, LifeSquare, KitchenSquare

In [ ]:
train_df.loc[train_df['Square'] < 5, 'Square'] = 7

In [ ]:
condition = (train_df['KitchenSquare'] > train_df['KitchenSquare'].quantile(.975))

train_df.loc[condition, 'KitchenSquare'] = train_df['KitchenSquare'].median()

train_df.loc[train_df['KitchenSquare'] < 3, 'KitchenSquare'] = 3

### Floor, HouseFLoor

In [ ]:
train_df['HouseFloor'].sort_values().unique()

In [ ]:
# Обрабатываем экземпляры у которых этаж квартиры выше этажности дома
train_df.loc[(train_df['Floor'] > train_df['HouseFloor']), 'Floor'] = train_df['HouseFloor'].median()

#train_df.loc[X['HouseFloor'] > 50, 'HouseFloor'] = train_df['HouseFloor'].median()

# Обрабатываем нулевой этаж дома
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor'] = train_df['HouseFloor'].median()

### HouseYear

In [ ]:
from datetime import datetime
date_t=datetime.now().year

In [ ]:
train_df['HouseYear'].sort_values(ascending=False)

In [ ]:
train_df.loc[train_df['HouseYear'] > 2020, 'HouseYear'] = 2011 

### 5.2. Номинативные признаки <a class='anchor' id='out_nan_mat'>

In [ ]:
# Заменим буквенные значения на числовые
litera = {'A': 0, 'B': 1}

train_df['Ecology_2'] = train_df['Ecology_2'].replace(litera)
train_df['Ecology_3'] = train_df['Ecology_3'].replace(litera)
train_df['Shops_2'] = train_df['Shops_2'].replace(litera)
train_df.head()

## 6. Классы подготовки данных <a class='anchor' id='data_prepare'>

In [ ]:
class DataPreprocessing:
        """Подготовка исходных данных"""

        def __init__(self):
            """Параметры класса"""
            self.medians=None
            self.kitchen_square_quantile = None
            self.binary_to_numbers = None


        def fit(self, X):
            """Сохранение статистик"""       
            self.medians = X.median()
            self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
            


        def transform(self, X):
            """Трансформация данных"""

            # Rooms
            X.loc[X['Rooms'] == 0, 'Rooms'] = 1
            X.loc[(train_df.Rooms >= 6), 'Rooms'] = X['Rooms'].median()
            
            # Square
            X.loc[X['Square'] < 5, 'Square'] = 7
            
            # Floor, HouseFLoor
            X.loc[(X['Floor'] > X['HouseFloor']), 'Floor'] = X['HouseFloor']
            X.loc[train_df['HouseFloor'] == 0, 'HouseFloor'] = X['HouseFloor'].median()

            # Square
            X.loc[X['Square'] < 5, 'Square'] = 7

            # KitchenSquare
            condition = (X['KitchenSquare'] > X['KitchenSquare'].quantile(.975))

            X.loc[condition, 'KitchenSquare'] = X['KitchenSquare'].median()

            X.loc[train_df['KitchenSquare'] < 3, 'KitchenSquare'] = 4

            # HouseYear
            X.loc[train_df['HouseYear'] > 2020, 'HouseYear'] = 2011
            
            # LifeSquare
            X['LifeSquare_nan'] = X['LifeSquare'].isna() * 1
            
            condition = (X['LifeSquare'].isna()) & (~X['Square'].isna()) & (~X['KitchenSquare'].isna())
        
            X.loc[condition, 'LifeSquare'] = X.loc[condition, 'Square'] - X.loc[condition, 'KitchenSquare']
            
            X.loc[(X['Social_3'] > 10), 'Social_3'] = self.medians['Social_3']
            
            # Healthcare_1
            if 'Healthcare_1' in X.columns:
                X.drop('Healthcare_1', axis=1, inplace=True)
            
            return X
            
class DataDummies():
    
    def __init__(self):
        self.binary_to_numbers = None
    
    
    def fit(self):
        
        self.binary_to_numbers = {'A': 0, 'B': 1}
        
        
    def transform(self, X):
        
        X['Ecology_2'] = X['Ecology_2'].map(self.binary_to_numbers)
        X['Ecology_3'] = X['Ecology_3'].map(self.binary_to_numbers)
        X['Shops_2'] = X['Shops_2'].map(self.binary_to_numbers)
        
        return X
    
    
    def floor_to_cat(self, X):
        X['floor_cat'] = 0
        X.loc[X['Floor'] <= 3, 'floor_cat'] = 1
        X.loc[(X['Floor'] > 3) & (X['Floor'] <= 5), 'floor_cat'] = 2
        X.loc[(X['Floor'] > 5) & (X['Floor'] <= 12), 'floor_cat'] = 3
        X.loc[(X['Floor'] > 12) & (X['Floor'] <= 20), 'floor_cat'] = 4
        X.loc[X['Floor'] > 20, 'floor_cat'] = 5
        X['floor_cat'].fillna(-1, inplace=True)
        
        return X
     
        
    def age_to_cat(self, X):
        X['age_cat'] = 0
        X.loc[(date_t - X['HouseYear']) <= 5, 'age_cat'] = 1
        X.loc[((date_t - X['HouseYear']) > 5) & ((date_t - X['HouseYear']) <= 10), 'age_cat'] = 2
        X.loc[((date_t - X['HouseYear']) > 10) & ((date_t - X['HouseYear']) <= 25), 'age_cat'] = 3
        X.loc[(date_t - X['HouseYear']) > 25, 'age_cat'] = 4
        X['age_cat'].fillna(-1, inplace=True)
        
        return X
    
    
    def ecology_to_cat(self, X):
        X['ecology_cat'] = 0
        X.loc[X['Ecology_1'] <= 0.1, 'ecology_cat'] = 1
        X.loc[(X['Ecology_1'] > 0.1) & (X['Ecology_1'] <= 0.2), 'ecology_cat'] = 2
        X.loc[(X['Ecology_1'] > 0.2) & (X['Ecology_1'] <= 0.3), 'ecology_cat'] = 3
        X.loc[(X['Ecology_1'] > 0.3) & (X['Ecology_1'] <= 0.4), 'ecology_cat'] = 4
        X.loc[X['Ecology_1'] > 0.4, 'ecology_cat'] = 5
        X['ecology_cat'].fillna(-1, inplace=True)

        return X

## 7. Разбиение на train и test <a class='anchor' id='split'>

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)

# Создадим объекты классов и проинициализируем переменные
preprocessor = DataPreprocessing()
preprocessor.fit(test_df)

features_gen = DataDummies()
features_gen.fit()

# Подготовим данные
train_df = preprocessor.transform(train_df)
train_df = features_gen.transform(train_df)

test_df = preprocessor.transform(test_df)
test_df = features_gen.transform(test_df)

# Отделим целевую переменную
X = train_df.drop('Price', axis=1)
y = train_df['Price']

# Создадим тренировочные и валидацонные наборы
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=21)

# Посмотрим на размерности и пропуски в данных
print(X_train.shape, X_valid.shape, test_df.shape)
print(X_train.isna().sum().sum(), 
      X_valid.isna().sum().sum(),
      test_df.isna().sum().sum())

## 8. Построение модели <a class='anchor' id='modeling'>

### Обучение модели

In [ ]:
gbr = GradientBoostingRegressor(criterion='mse',
                                     max_depth=6,
                                     random_state=42,
                                     min_samples_leaf=35,
                                     n_estimators=140)
gbr.fit(X_train, y_train)

### Оценка модели

In [ ]:
y_train_preds = gbr.predict(X_train)
y_test_preds = gbr.predict(X_valid)

evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

In [ ]:
cv_score = cross_val_score(gbr, X_train, y_train, scoring='r2', cv=KFold(n_splits=3, shuffle=True, random_state=21))
cv_score

In [ ]:
feature_importances = pd.DataFrame(zip(X_train.columns, gbr.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

## 9. Прогнозирование на тестовом датасете <a class='anchor' id='prediction'>

In [ ]:
submit = pd.read_csv('/kaggle/input/real-estate-price-prediction-moscow/sample_submission.csv')
submit.head()

In [ ]:
predictions = gbr.predict(test_df)
predictions

In [ ]:
submit['Price'] = predictions
submit.head()

In [ ]:
submit.to_csv('gbr_submit.csv', index=False)